In [1]:
import os
import pandas as pd
import numpy as np
import numpy.polynomial.polynomial as nppoly
from data_loader import PortLoader
from analyzer import ReturnAnalyzer

### Data Loading

In [2]:
# from datetime import date
import yfinance as yf
bist100 = yf.Ticker('XU100.IS')
bist50 = yf.Ticker('XU050.IS')

bist100_df = yf.download('XU100.IS', start='2022-01-01')
bist50_df = yf.download('XU050.IS', start='2022-01-01')
bist100_df.tail()

# TO-DO: Find API that have BIST50 Index data.

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-16,5211.799805,5259.000000,5154.600098,5214.299805,5214.299805,6291498000
2022-12-19,5270.600098,5404.000000,5267.700195,5391.899902,5391.899902,7195195300
2022-12-20,5408.799805,5445.899902,5344.299805,5419.000000,5419.000000,6991291700
2022-12-21,5449.299805,5491.100098,5412.700195,5429.100098,5429.100098,5943969000
2022-12-22,5462.180176,5515.669922,5398.430176,5447.229980,5447.229980,0


In [3]:
bist100_df['Daily Return'] = (bist100_df['Close'] / bist100_df['Close'].shift(1)) -1
bist50_df['Daily Return'] = (bist50_df['Close'] / bist50_df['Close'].shift(1)) -1 
bist100_df = bist100_df.dropna()
bist50_df = bist50_df.dropna()
bist100_df.head()
bist100_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2022-12-16,5211.799805,5259.000000,5154.600098,5214.299805,5214.299805,6291498000,0.004914
2022-12-19,5270.600098,5404.000000,5267.700195,5391.899902,5391.899902,7195195300,0.034060
2022-12-20,5408.799805,5445.899902,5344.299805,5419.000000,5419.000000,6991291700,0.005026
2022-12-21,5449.299805,5491.100098,5412.700195,5429.100098,5429.100098,5943969000,0.001864
2022-12-22,5462.180176,5515.669922,5398.430176,5447.229980,5447.229980,0,0.003339


### Parsing Files

In [4]:
os.getcwd()

'c:\\Users\\ConquerV\\Documents\\2022-23Fall\\rsm2013\\quant'

In [5]:
# This can be changed to the data directory for weights
os.chdir('c:/Users/ConquerV/Documents/2022-23Fall/rsm2013/quant/data/weight_1')
cwd = os.getcwd()

# store paths to all data xml files in a list for iteration
w1_path = [os.path.join(cwd, f) for f in os.listdir(cwd)]
os.path.basename(w1_path[0])

'longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_win_l_10_win_s_8_es_l_2_es_s_2.csv'

In [6]:
w1_port = PortLoader(w1_path)
w1_port.port_params.head()

Loading Portfolio Data...
Function 'load_port' executed in 0.0071s


,port_name,long_short,holding_period,training_weight,proportion of short,scale of returns_long,scale of returns_short
0,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
1,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
2,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
3,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
4,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01


### Portfolio Analysis

In [7]:
def sharpe_ratio(port, rf=0):
    """
    This function returns the daily sharpe ratio of a portfolio
    as (E[R] - rf)/Std(R)
    """
    port_ret = pd.DataFrame(port['Daily Return'].dropna())
    mean = port_ret.mean()
    std = port_ret.std()
    sharpe = (mean - rf)/std
    return float(sharpe)

In [172]:
def jensen_alpha(port, benchmark, rf=0):
    """
    Returns excess return of a portfolio according to
    Rp - (rf + beta x (rm - rf))
    """
    port_ret = port['Daily Return']
    mkt_ret  = benchmark['Daily Return']
    port_beta, port_alpha = np.polyfit(benchmark['Daily Return'], port['Daily Return'], 1)

    port_ret_mean = port_ret.mean()
    mkt_ret_mean = mkt_ret.mean()

    port_alpha = port_ret_mean - (rf + port_beta * (mkt_ret_mean - rf))
    port['Excess Return'] = port['Daily Return'] - benchmark['Daily Return']
    return port_alpha, port_beta

In [175]:
def port_analysis(path_list: list, scales: pd.Series, mkt_port: pd.DataFrame):
    """This performs some basic portfolio analysis"""
    temp = {'Portfolio Beta': [],
            'Sharpe':[],
            'Alpha':[]
            }
    for i in range(len(path_list)):
        filepath = path_list[i]
        filename = os.path.basename(filepath)
        if 'param' in filename and 'long' in filename:
            scale = scales[i]
            port = pd.read_csv(filepath)
            port = port.dropna()
            # port['Daily Return'] = port['Daily Return'].apply(lambda x: x/scale)

            # Benchmark
            start, end = port['Dates'].iloc[0], port['Dates'].iloc[-1]
            mkt_port = mkt_port[mkt_port.index.to_series().between(start, end)]
            mkt_port = mkt_port.dropna()

            # Report Metrics
            port_sharpe = sharpe_ratio(port)
            port_alpha, port_beta = jensen_alpha(port, mkt_port)
            
            temp['Portfolio Beta'].append(port_beta)
            temp['Sharpe'].append(port_sharpe)
            temp['Alpha'].append(port_alpha)

    df_metric = pd.DataFrame(data=temp)
    return df_metric
    

In [8]:
analyzer = ReturnAnalyzer(w1_path)
df_metric = analyzer.evaluate()
df_metric.head()


Loading Portfolio Data...
Function 'load_port' executed in 0.0060s
[*********************100%***********************]  1 of 1 completed
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' executed in 0.0162s
Function 'sharpe_ratio' executed in 0.0010s
Function 'jensen_alpha' executed in 0.0010s
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' executed in 0.0000s
Function 'sharpe_ratio' executed in 0.0010s
Function 'jensen_alpha' executed in 0.0020s
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' executed in 0.0020s
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' executed in 0.0000s
Function 'sharpe_ratio' executed in 0.0010s
Function 'jensen_alpha' executed in 0.0030s
Function 'sharpe_ratio' executed in 0.0020s
Function 'jensen_alpha' executed in 0.0011s
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' executed in 0.0092s
Function 'sharpe_ratio' executed in 0.0000s
Function 'jensen_alpha' exec

,Params,Beta,Sharpe,Alpha
0,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.732261,0.392174,0.003784
1,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.742263,0.377382,0.003716
2,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.685482,0.354982,0.003012
3,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.737537,0.400035,0.004195
4,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.733396,0.398851,0.004300


In [1]:
sample_port =  pd.read_csv(w1_path[4])
sample_port = sample_port.dropna()
sharpe = sharpe_ratio(sample_port)

print("The sharpe of a given portfolio is", sharpe)

long_bm = bist100_df

start, end = sample_port['Dates'].iloc[0], sample_port['Dates'].iloc[-1]
long_bm = long_bm[long_bm.index.to_series().between(start, end)]

print(sample_port['Daily Return'].head())
sample_port['Daily Return'] = sample_port['Daily Return'].apply(lambda x: x/0.1)
print(sample_port['Daily Return'].head())

# port_ret = sample_port['Daily Return'].cumprod()
# mkt_ret  = long_bm['Daily Return'].cumprod()
# print(port_ret.var(), mkt_ret.var(), port_ret.cov(mkt_ret))
mkt_ret.head()

beta, alpha = np.polyfit(sample_port['Daily Return'], long_bm['Daily Return'], 1)
print(beta, alpha)
# long_bm.head()
# print(mkt_ret.mean(), port_ret.mean())

NameError: name 'pd' is not defined

In [93]:
sample_port.dropna().head()

,Dates,Stocks,Number of Stocks,Energy,Execution Time,Portfolio Value,Daily Return,Cumulative Return,Daily Long Return,Cumulative Long Return,Daily Short Return,Cumulative Short Return
1,2022-06-07,"{'long AKBNK.IS': 284.67259866162317, 'long AK...",36,-0.043776,0.743596,10065.320630,0.006532,0.006532,0.006532,0.024821,0.0,0.0
2,2022-06-08,"{'long AKSA.IS': 257.4085879060134, 'long ALBR...",38,-0.045759,0.733187,9606.962069,-0.045538,-0.039304,-0.045538,-0.021847,0.0,0.0
3,2022-06-09,"{'long AKSA.IS': 904.7016098231707, 'long BERA...",11,-0.012133,0.596324,9774.116147,0.017399,-0.022588,0.017399,-0.004828,0.0,0.0
4,2022-06-10,"{'long AKSA.IS': 836.2072522290761, 'long BERA...",12,-0.014375,0.656900,9855.408338,0.008317,-0.014459,0.008317,0.003449,0.0,0.0
5,2022-06-13,"{'long AKSA.IS': 900.7923316826433, 'long BERA...",11,-0.013678,0.663277,9731.881516,-0.012534,-0.026812,-0.012534,-0.009128,0.0,0.0


In [13]:
class Dataloader:
    """A class that abstracts the loading process of portfolio with corresponding training weights

    Attributes:
        data_path: 
            relative path to the data folder, e.g. './data'
        port_df:
            a dataframe that stores all the portfolio of the given data path
    """
    def __init__(self, data_path: str):
        """Initialize Dataloader with data_path, the relative path to the data folder"""
        self.data_path = data_path

    def load_port(self, port_filename: str) -> pd.DataFrame:
        """ Load return data from specified file and convert it to a multiindexed serie

        Args: 
            port_filename: portfolio holdings given a parameter file used in data folder, e.g. 'long_10_day_param_test_w.csv'

        Return:
            A pandas dataframe object with multiindex (date, asset)
        """
        port = pd.read_csv(self.data_path + port_filename)
        daily_ret = port.columns[6].astype(float)

        # Pad the asset code and add corresponding prefixes 
        
        # Combine all indexes
        
        # Transform the table into desired shape

        temp = {'long_short': [],
            'holding_period': [],
            'training_weight': [],
            'proportion of short': [],
            'scale of returns_long': [],
            'scale of returns_short': []
            }
    
        for filepath in path_list:
            filename = os.path.basename(filepath)
            if 'param' in filename and 'results' not in filename:
                params = filename.split('_')
                temp['long_short'].append(int('longs' in params[0]))
                temp['holding_period'].append(float(params[1]))
                temp['training_weight'].append(params[6])
                temp['proportion of short'].append(float(params[8])/10)
                temp['scale of returns_long'].append(float(params[10])/100)
                temp['scale of returns_short'].append(float(params[12])/100)
                # temp['lookback period_long'].append(float())
                # TO-DO: add remaining parameters to the portfolio
        df = pd.DataFrame(data=temp)

        return daily_ret.to_frame("1D")


In [126]:
class Backtest:
    """A class that abstracts the loading process of portfolio with corresponding training weights

        Attributes:
            data_path: 
                relative path to the data folder, e.g. './data'
    """
    def __init__(self, port: pd.DataFrame):
        """Initialize Dataloader with data_path, the relative path to the data folder"""
        self.port = port

    def port_sharpe(self, index_data_file: str):
        pass
